# Урок 6
## Взаимосвязь величин. Показатели корреляции. Корреляционный анализ. Проверка на нормальность

### Задача 1
#### Условие:
Даны значения величины заработной платы заемщиков банка (`salary`) и значения их поведенческого кредитного скоринга (`scoring`):

`salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]`

`scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]`

Используя только встроенные питоновские функции и структуры данных (т.е. без библиотек numpy, pandas и др.) найдите:
1. ковариацию (смещённую и несмещённую) этих двух величин,
2. коэффициент корреляции Пирсона.

Можно затем посчитать те же значения с использованием библиотек, чтобы проверить себя.

#### Решение:
##### Пункт 1:
$$cov(X,Y) = \dfrac{1}{n - 1} \displaystyle\sum_{i = 1}^n \left( (x_i - \overline{X} )( y_i - \overline{Y} )\right)$$


In [1]:
def mean(arr: list) -> float:
    return sum(arr) / len(arr)

In [2]:
def cov(X: list, Y: list, is_unbiased: bool = False) -> float:
    mean_X = mean(X)
    mean_Y = mean(Y)
    return sum((x - mean_X)*(y - mean_Y) \
               for x, y in zip(X, Y))/(len(X) - int(is_unbiased))


In [3]:
salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]

In [4]:
cov(salary, scoring, is_unbiased=True)

10175.377777777778

In [5]:
cov(salary, scoring, is_unbiased=False)

9157.84

##### Пункт 2:

$$Pirson(X,Y) = \dfrac{cov(X,Y)}{\sigma_X \sigma_Y}$$

In [6]:
import math

def variance(data: list, is_unbiased: bool = False) -> float:
    data_mean = mean(data)
    return sum((elem - data_mean) ** 2 for elem in data) / (len(data) - int(is_unbiased))

def std(data: list, is_unbiased: bool = False) -> float:
    return math.sqrt(variance(data, is_unbiased))


In [7]:
def Pirson(X: list, Y: list) -> float:
    return cov(X, Y) / (std(X) * std(Y))

In [8]:
Pirson(salary, scoring)

0.8874900920739162

#### Задача 2
##### Условие: 
Проведите тест на значимость коэффициента корреляции Пирсона, найденного в предыдущей задаче. 

Что для этого нужно знать:
* Нулевая гипотеза: реальный коэффициент корреляции равен `0`. Альтернативная гипотеза двухсторонняя.
* Статистика: `t = r * sqrt(n - 2) / sqrt(1 - r ** 2)`, где `r` - коэффициент корреляции Пирсона, посчитанный по выборке.
* В предположении верности нулевой гипотезы эта статистика имеет распределение Стьюдента с параметром `df = n - 2`.

##### Решение:
Пусть $\alpha = 0.05$

Найдем квантили:

In [9]:
from scipy import stats

alpha = 0.05
n = len(salary)

t_1 = stats.t.ppf(alpha / 2, df=n - 2)
t_2 = stats.t.ppf(1 - alpha / 2, df=n - 2)

t_1, t_2

(-2.306004135033371, 2.3060041350333704)

Критическая область:

$\Omega_\alpha = (-\infty, -2.306) \cup (2.306, +\infty)$

In [10]:
r = Pirson(salary, scoring)

t = r * math.sqrt(n - 2) / math.sqrt(1 - r ** 2)
t

5.447168150485575

In [11]:
t < -2.306 or t > 2.306

True

Статистика попала в критическую область, сл-но нулевая гипотеза отвергается, т.е. значимость между выборками значима

#### Задача 3
##### Условие:

Измерены значения IQ выборки студентов, обучающихся в местных технических вузах:

`131, 125, 115, 122, 131, 115, 107, 99, 125, 111`

Известно, что в генеральной совокупности IQ распределен нормально. 

Найдите доверительный интервал для математического ожидания с надежностью `0.95`.

##### Решение:

Дисперсия неизвестна, сл-но воспользуемся формулой:

$P \left( \overline{X} + t_{{\alpha / 2}, n - 1} \cdot \dfrac{\sigma_X}{\sqrt{n}} \leq \mu \leq \overline{X} + t_{{1 - \alpha / 2}, n - 1} \cdot \dfrac{\sigma_X}{\sqrt{n}} \right) = p = 0.95$

In [12]:
import numpy as np

data = np.array([131, 125, 115, 122, 131, 115, 107, 99, 125, 111])
p = 0.95
alpha = 1 - p

In [13]:
n = data.size
mean = data.mean()
sigma = data.std(ddof=1)

n, mean, sigma

(10, 118.1, 10.54566788359614)

Найдем квантили:

In [14]:
t_1 = stats.t.ppf(alpha / 2, df=n - 1)
t_2 = stats.t.ppf(1 - alpha / 2, df=n - 1)

t_1, t_2

(-2.2621571627409915, 2.2621571627409915)

In [15]:
(mean + t_1 * sigma / math.sqrt(n), mean + t_2 * sigma / math.sqrt(n))

(110.55608365158724, 125.64391634841274)